In [151]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [1]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [2]:
import pandas as pd
import numpy as np
import spacy
import nltk
from nltk.tokenize import word_tokenize

### Torch Packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence

In [152]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
# index 314 has no text
df_train.dropna(inplace=True)

### Preprocessing
* Lowercase - possible since the predicted sentiment text and the selected_text will be lowercase when computing metric.
* punctuation - keep the punctuation given that the submission file states that need to be quoted and complete. 
* Numericalize - Turn each token into its corresponding tokens.

In [5]:
def preprocessing(sentence):
    """
    This function will preprocess the input sentence sequence to avoid any further preprocessing
    downstream.
    """
    return sentence.lower().split()

In [6]:
# lowercasing all the text and turning them into a list of tokens for text and selected text in the training set
df_train['text'] = df_train['text'].progress_apply(preprocessing)
df_train['selected_text'] = df_train['selected_text'].progress_apply(preprocessing)
df_test['text'] = df_test['text'].progress_apply(preprocessing)

In [80]:
# making the sentiment to a variable
df_train['sentiment'] = df_train['sentiment'].astype('category')
df_train['code'] = df_train['sentiment'].cat.codes
df_test['sentiment'] = df_test['sentiment'].astype('category')
df_test['code'] = df_test['sentiment'].cat.codes

In [84]:
df_train.head()

,textID,text,selected_text,sentiment,code
0,cb774db0d1,"[i`d, have, responded,, if, i, were, going]","[i`d, have, responded,, if, i, were, going]",neutral,1
1,549e992a42,"[sooo, sad, i, will, miss, you, here, in, san,...","[sooo, sad]",negative,0
2,088c60f138,"[my, boss, is, bullying, me...]","[bullying, me]",negative,0
3,9642c003ef,"[what, interview!, leave, me, alone]","[leave, me, alone]",negative,0
4,358bd9e861,"[sons, of, ****,, why, couldn`t, they, put, th...","[sons, of, ****,]",negative,0


In [8]:
def unique_words(data):
    """
    Find the number of unique words in the training set.
    """
    words = set()
    for text in data:
        for word in text:
            words.add(word)
    return words

In [13]:
unq_words = unique_words(df_train.text)
len(unq_words)

45433

In [122]:
def create_vocabulary(words):
    vocab2idx, idx = {}, 5
    vocab2idx['<pad>'] = 0
    vocab2idx['<unk>'] = 1
    vocab2idx['<sos>'] = 3
    vocab2idx['<eos>'] = 4
    
    for word in words:
        vocab2idx[word] = idx
        idx += 1
    return vocab2idx

In [123]:
vocab2idx = create_vocabulary(unq_words)
len(vocab2idx)

45437

### Numericalize

In [131]:
def encoding_with_no_padding(sentence, vocab2idx):
    numericalize = [vocab2idx['<sos>']]
    for token in sentence:
        numericalize.append(vocab2idx.get(token, vocab2idx['<unk>']))
    numericalize.append(vocab2idx['<eos>'])
    return numericalize

In [132]:
class tweetDataset(Dataset):
    def __init__(self, data, vocab2idx):
        self.X = [encoding_with_no_padding(x, vocab2idx) for x in data['text']]
        self.y = [encoding_with_no_padding(y, vocab2idx) for y in data['selected_text']]
        self.sentiment = data.code.values
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.LongTensor(self.X[idx]), torch.LongTensor(self.y[idx]), self.sentiment[idx]

In [133]:
# sanity check
train_ds = tweetDataset(df_train, vocab2idx)

In [134]:
x, y, s = train_ds[5]
x

tensor([    3, 29460, 23733, 39359, 12376, 22814,  8494, 17882, 44452,  8516,
        20939, 41623,  5020,     4])

In [136]:
s

1

In [162]:
def collate(batch):
    (X, y, s) = zip(*batch)
    x_len = [len(x) for x in X]
    x_pad = pad_sequence(X, batch_first=True, padding_value=0)
    y_pad = pad_sequence(y, batch_first=True, padding_value=0)
    return x_pad, x_len, y_pad, torch.LongTensor(s)

In [139]:
# sanity check
batch_size = 3
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate)
x,lengths, y, s = next(iter(train_dl))

### Seq2Seq

In [174]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, sentiment):
        super().__init__()
        self.vocabs = nn.Embedding(vocab_size, emb_dim)
        self.sentiment = nn.Embedding(sentiment, hidden_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(.5)
        
    def forward(self, x, lengths, sentiment):
        x = self.dropout(self.vocabs(x))
        x_pack = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        out, (hidden, cell) = self.lstm(x)
        return hidden[-1] + self.sentiment(sentiment), cell

In [121]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim):
        super().__init__()
        self.vocabs = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, x, hidden, cell):
        x = self.vocabs(x)
        output, (hidden, cell) = self.lstm(x, (hidden, cell))
        return self.linear(hidden[-1]), hidden, cell  #NOTE: hidden[-1] returns everything within that batch

In [144]:
def train_model(encoder, decoder, enc_optimizer, dec_optimizer, train_dl, epochs=10):
    for epoch in tqdm(range(epochs)):
        total_loss, total = 0, 0
        for x, lengths, y, s in train_dl:
            loss = train_batch(encoder, decoder, enc_optimizer, dec_optimizer, x.to(device), y.to(device), s.to(device), lengths)
            total_loss += loss*x.size(0)
            total += x.size(0)
        print(f"Epoch: {epoch+1} Training Loss: {loss/total_loss:.3f}")

In [179]:
def train_batch(encoder, decoder, enc_optimizer, dec_optimizer, x, y, s, lengths,
                teacher_forcing_ratio=0.5):
    encoder.train()
    decoder.train()
    enc_optimizer.zero_grad()
    dec_optimizer.zero_grad()
    loss = 0
    
    # **ENCODER**
    hidden, cell = encoder(x, lengths, s)  # passing both the sequence and the sentiment
    hidden = hidden.unsqueeze(0)
    
    # **DECODER**
    batch_target_length = y.size(1)  # NOTE: the length of the selected text
    decoder_input = x[:, 0].unsqueeze(1)
    
    for idx in range(1, batch_target_length):
        output, hidden, cell = decoder(decoder_input, hidden, cell)
        y_idx = y[:, idx]
        loss += F.cross_entropy(output, y_idx, ignore_index=0)
        teacher_force = True if np.random.uniform() > teacher_forcing_ratio else False
        if teacher_force:
            decoder_input = y_idx
        else:
            decoder_input = output.argmax(dim=1).unsqueeze(1)
    
    # updating the gradient
    loss.backward()
    enc_optimizer.step()
    dec_optimizer.step()
    return loss.item()

In [175]:
encoder = Encoder(len(vocab2idx), 32, 64, len(df_train.code.unique())).to(device)
decoder = Decoder(len(vocab2idx), 64, 64).to(device)
enc_optimizer = optim.Adam(encoder.parameters(), lr=3e-4)
dec_optimizer = optim.Adam(decoder.parameters(), lr=3e-4)

In [172]:
batch_size = 256
train_ds = tweetDataset(df_train, vocab2idx)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate)

In [180]:
train_model(encoder, decoder, enc_optimizer, dec_optimizer, train_dl)

/data/dquan3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


RuntimeError: input must have 3 dimensions, got 2

In [165]:
# sanity check
batch_size = 3
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate)
x,lengths, y, s = next(iter(train_dl))

In [169]:
enc = Encoder(len(vocab2idx), 32, 64, len(df_train.code.unique()))

In [185]:
x[:, 0]

tensor([3, 3, 3])

In [184]:
o, h, c = enc(x[:, 0], lengths, s)

RuntimeError: input must have 3 dimensions, got 2